In [1]:
!pip install -q langchain chromadb sentence-transformers transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 148.9 

In [2]:
import os

os.makedirs("data", exist_ok=True)

refund_policy = """
Customers can request a refund within 14 days of purchase.
Refunds are issued only if the product is unused and in original packaging.
Digital products are non-refundable once accessed.
"""

cancellation_policy = """
Orders can be cancelled within 24 hours of placing the order.
Once shipped, orders cannot be cancelled.
Subscription plans can be cancelled anytime, but no partial refunds are provided.
"""

shipping_policy = """
Standard shipping takes 5-7 business days.
Express shipping takes 2-3 business days.
Shipping delays may occur due to weather or operational issues.
"""

with open("data/refund_policy.txt", "w") as f:
    f.write(refund_policy)
with open("data/cancellation_policy.txt", "w") as f:
    f.write(cancellation_policy)
with open("data/shipping_policy.txt", "w") as f:
    f.write(shipping_policy)

In [6]:
!pip install -q langchain langchain-community langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 132.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 609.9/609.9 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 37.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently 

In [7]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

docs = []
for file in os.listdir("data"):
    loader = TextLoader(f"data/{file}")
    docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=40
)

chunks = text_splitter.split_documents(docs)

print(f"Total chunks created: {len(chunks)}")


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


Total chunks created: 3


In [9]:
!pip install -q sentence-transformers chromadb

In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/tmp/ipython-input-2671871813.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="chroma_db"
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
)

In [12]:
from transformers import pipeline

llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=256
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [13]:
IMPROVED_PROMPT = """
You are a company policy assistant.

Rules:
- Answer ONLY using the provided context.
- If the answer is not present, respond exactly with:
  "The information is not available in the provided documents."
- Do NOT guess or add external information.
- Keep the answer concise and factual.

Context:
{context}

Question:
{question}

Answer:
"""

In [14]:
def answer_question(question, prompt_template):
    retrieved_docs = retriever.get_relevant_documents(question)

    if not retrieved_docs:
        return "The information is not available in the provided documents."

    context = "\n".join(doc.page_content for doc in retrieved_docs)

    prompt = prompt_template.format(
        context=context,
        question=question
    )

    response = llm(prompt)[0]["generated_text"]
    return response

In [17]:
def answer_question(question, prompt_template):
    retrieved_docs = retriever.invoke(question)

    if not retrieved_docs:
        return "The information is not available in the provided documents."

    context = "\n".join(doc.page_content for doc in retrieved_docs)

    prompt = prompt_template.format(
        context=context,
        question=question
    )

    response = llm(prompt)[0]["generated_text"]
    return response

questions = [
    "How many days do I have to request a refund?",
    "Can I cancel an order after it is shipped?",
    "How long does express shipping take?"
]

for q in questions:
    print("Q:", q)
    print("A:", answer_question(q, IMPROVED_PROMPT))
    print("-" * 50)

Q: How many days do I have to request a refund?
A: 14 days.
--------------------------------------------------
Q: Can I cancel an order after it is shipped?
A: Orders can be cancelled within 24 hours of placing the order. Once shipped, orders cannot be cancelled.
--------------------------------------------------
Q: How long does express shipping take?
A: 2-3 business days.
--------------------------------------------------


In [18]:
print(
    answer_question(
        "Do you offer international shipping?",
        IMPROVED_PROMPT
    )
)

Yes.


# LETS EVALUATE WITH SET OF QUESTIONS

In [19]:
evaluation_questions = [
    "Are digital products refundable?",
    "Can a subscription be cancelled anytime?",
    "Is there a partial refund for subscriptions?",
    "Do you provide student discounts?",
    "What causes shipping delays?"
]

for q in evaluation_questions:
    print("Q:", q)
    print("A:", answer_question(q, IMPROVED_PROMPT))
    print("=" * 50)

Q: Are digital products refundable?
A: Digital products are non-refundable once accessed.
Q: Can a subscription be cancelled anytime?
A: Subscription plans can be cancelled anytime, but no partial refunds are provided. Customers can request a refund within 14 days of purchase. Refunds are issued only if the product is unused and in original packaging. Digital products are non-refundable once accessed. Standard shipping takes 5-7 business days. Express shipping takes 2-3 business days. Shipping delays may occur due to weather or operational issues.
Q: Is there a partial refund for subscriptions?
A: Subscription plans can be cancelled anytime, but no partial refunds are provided.
Q: Do you provide student discounts?
A: No.
Q: What causes shipping delays?
A: weather or operational issues.
